In [1]:
import glassflow

In [2]:
import os

In [3]:
client = glassflow.GlassFlowClient(
    personal_access_token=os.getenv("GLASSFLOW_TOKEN")
)

In [4]:
list_spaces = client.list_spaces()

space_name = "main"
for s in list_spaces.spaces:
    if s["name"] == space_name:
        space = glassflow.Space(
            personal_access_token=client.personal_access_token,
            id=s["id"], 
            name=s["name"]
        )
        break
else:
    space = client.create_space(name=space_name)

print(f"Space \"{space.name}\" with ID: {space.id}")

Space "main" with ID: bdecbdc6-af8a-4e22-a5da-3c7d921d7292


In [5]:
%pycat transform.py

import os 
from phi.agent import Agent
from pydantic import BaseModel
from pydantic.fields import Field
from openai import OpenAI
from phi.model.openai import OpenAIChat
from supabase import create_client, Client


def handler(data, log):
    supabase: Client = create_client(os.getenv("SUPABASE_URL"), os.getenv("SUPABASE_KEY"))
    result, parsed = get_ai_results(
        base64_image=data["base64_image"],
        plant=data["plant"],
        location=data["location"],
        weather=data["weather"],
        temperature=data["temperature"]
    )
    

    data = {
        "image_id": data["image_id"],
        "sensor_id": data["sensor_id"], 
        "plantation_id": data["plantation_id"],  
        "prediction_details": result, 
        "pest": parsed.pest,
        "weed": parsed.weed,
        "disease": parsed.disease
    }

    response = supabase.table("predictions").insert(data).execute()

    return result


def get_ai_results(base64_image, plant, location, weather, temperature):

In [6]:
with open("requirements.txt") as f:
    requirements_txt = f.read()
print(requirements_txt)

glassflow>=2.0.6
ipykernel>=6.29.5
openai>=1.55.3
opencv-python>=4.10.0.84
phidata>=2.5.33
pydantic>=2.10.2
python-dotenv>=1.0.1
supabase>=2.10.0


In [7]:
env_vars = [
  {
    "name": "OPENAI_API_KEY",
    "value": os.getenv("OPENAI_API_KEY")
  },
  {
    "name": "SUPABASE_URL",
    "value": os.getenv("SUPABASE_URL")
  },
  {
    "name": "SUPABASE_KEY",
    "value": os.getenv("SUPABASE_KEY")
  }
]

In [ ]:
pipeline_name = "agro-image-analysis"

pipeline = client.create_pipeline(
    name=pipeline_name, 
    transformation_file='transform.py',
    space_id=space.id, 
    env_vars=env_vars,
    requirements=requirements_txt,
    source_kind="webhook",
    source_config={
        "image_id": "<your image id>",
        "base64_image": "<your encoded image>",
        "plant": "<your plant name>",
        "location": "<your location>",
        "weather": "<your weather>",
        "temperature": "<your temperature>",
        "sensor_id": "<your sensor id>",
        "plantation_id": "<your plantation id>",
    }, 
)
print("Pipeline ID:", pipeline.id)

In [9]:
print("Pipeline is deployed!") 
print("Pipeline Id = %s" % (pipeline.id))
print("Pipeline URL %s "% f"https://app.glassflow.dev/pipelines/{pipeline.id}")

Pipeline is deployed!
Pipeline Id = 55800b46-8354-44ae-93ed-3478f27cc2d6
Pipeline URL https://app.glassflow.dev/pipelines/55800b46-8354-44ae-93ed-3478f27cc2d6 


In [10]:
data_source = pipeline.get_source()

In [11]:
from glassflow import PipelineDataSource

In [11]:
import base64

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [12]:
enc_img = encode_image("../images/a-Wilt-in-Capsicum.jpg")

In [ ]:
source = PipelineDataSource(
	pipeline_id="",
	pipeline_access_token=""
)
# this is dummy data, format deosn't have to be same

data = {
    "image_id": 2,
    "base64_image": enc_img,
    "plant": "capscicum",
    "location": "Colombo",
    "weather": "31 celcius",
    "temperature": "27 celcius",
    "sensor_id": 234,
    "plantation_id": 1,
}
source.publish(data)

PublishEventResponse(content_type=None, status_code=200, raw_response=<Response [200]>, object=None)

In [23]:
enc_img

'...'

In [ ]:
from glassflow import PipelineDataSink
sink = PipelineDataSink(
	pipeline_id="",
	pipeline_access_token=""
)
response = sink.consume()
if response.status_code == 200:
	data = response.json()
	print("Consumed Data: ", data)

In [22]:
data

[{'image_id': 2,
  'base64_image': '...',
  'plant': 'capscicum',
  'location': 'Colombo',
  'weather': '31 celcius',
  'temperature': '27 celcius',
  'sensor_id': 234,
  'plantation_id': 1}]

In [30]:
sink.consume_failed()

ConsumeFailedResponse(content_type=None, status_code=204, raw_response=<Response [204]>, body=ConsumeFailedResponseBody(req_id='', receive_time='', event={}))